In [1]:
from transformers import AutoModel
import torchxrayvision as xrv
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

In [2]:
# Charger un modèle DenseNet pré-entraîné sur CheXpert
model = xrv.models.DenseNet(weights="densenet121-res224-chex")

# Vérification
print(model)

XRV-DenseNet121-densenet121-res224-chex


In [3]:
data_dir = "/home/zoidberg/Zoidberg/data/raw/chest_Xray/test"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Adapté à xrv.models.DenseNet
])

full_dataset = datasets.ImageFolder(data_dir, transform=transform)

In [4]:
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

In [5]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Exemple si vous avez les targets : 
targets = np.array(full_dataset.targets)

for train_idx, val_idx in kfold.split(np.arange(len(targets)), targets):
    train_subset = torch.utils.data.Subset(full_dataset, train_idx)
    val_subset = torch.utils.data.Subset(full_dataset, val_idx)
    # Entraînez ici votre modèle sur train_subset et évaluez sur val_subset

In [6]:
model = xrv.models.DenseNet(weights="densenet121-res224-chex")
for param in model.parameters():
    param.requires_grad = False

# Supposons que vous ajoutiez une tête binaire pour la pneumonie
classifier = nn.Sequential(
    nn.Linear(1024, 1),  # 1024 dépend de la sortie de DenseNet
    nn.Sigmoid()
)
model.op_threshs = None  # ← désactive la normalisation interne du modèle
model.op_norm = nn.Identity()  # Empêche la normalisation interne
model.classifier = classifier  # Ajoute la tête personnalisée

In [7]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = Adam(model.classifier.parameters(), lr=1e-4)

for epoch in range(10):
    model.train()
    for x, y in train_loader:
        x = x.to(device)
        y = y.float().unsqueeze(1).to(device)
        out = model(x)
        loss = F.binary_cross_entropy(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [8]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        preds = outputs.cpu().numpy()
        labels = y.cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels)

# Conversion pour AUC et autres
all_preds = np.array(all_preds).reshape(-1)
all_labels = np.array(all_labels).reshape(-1)

print("Nombre de batchs dans val_loader :", len(val_loader))
print("AUC:", roc_auc_score(all_labels, all_preds))
print("Accuracy:", accuracy_score(all_labels, all_preds > 0.5))
print("F1-score:", f1_score(all_labels, all_preds > 0.5))

NameError: name 'val_loader' is not defined